# Header

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import janitor
from sklearn.linear_model import LinearRegression
import os

exec(open("../header.py").read())

Header initialized


# Import

In [6]:
raw_data = pd.read_csv(raw_root("01-poetryfoundation/poetry_foundation.csv"))

In [7]:
raw_data.head()

,Unnamed: 0,Author,Title,Poetry Foundation ID,Content
0,0,Wendy Videlock,!,55489,"Dear Writers, I’m compiling the first in what ..."
1,1,Hailey Leithauser,0,41729,"Philosophic\nin its complex, ovoid emptiness,\..."
2,2,Jody Gladding,1-800-FEAR,57135,We'd like to talk with you about fear t...
3,3,Joseph Brodsky,1 January 1965,56736,The Wise Men will unlearn your name.\nAbove yo...
4,4,Ted Berrigan,3 Pages,51624,For Jack Collom\n10 Things I do Every Day\n\np...


In [8]:
raw_data.columns

Index(['Unnamed: 0', 'Author', 'Title', 'Poetry Foundation ID', 'Content'], dtype='object')

# Clean

In [9]:
data = raw_data\
    .drop('Unnamed: 0', axis = 1)\
    .clean_names()

In [10]:
data.head()

,author,title,poetry_foundation_id,content
0,Wendy Videlock,!,55489,"Dear Writers, I’m compiling the first in what ..."
1,Hailey Leithauser,0,41729,"Philosophic\nin its complex, ovoid emptiness,\..."
2,Jody Gladding,1-800-FEAR,57135,We'd like to talk with you about fear t...
3,Joseph Brodsky,1 January 1965,56736,The Wise Men will unlearn your name.\nAbove yo...
4,Ted Berrigan,3 Pages,51624,For Jack Collom\n10 Things I do Every Day\n\np...


## Train-validation-

## Bag of words

In [11]:
def extract_words_from_emails(emails):
    '''
    Purpose: Helper function for bag_of_words
    Input: emails
    Output: list of words that occur in more than threshold emails
    '''
    
    threshold = 30
    word_counts = {}
    
    for email in emails:
        for word in email:
            if word in word_counts:
                word_counts[word] += 1
            else:
                word_counts[word] = 1 
                
    filtered_word_counts = word_counts.copy()

    for i in word_counts:
        if filtered_word_counts[i] < threshold:
            filtered_word_counts.pop(i)
            
    return list(filtered_word_counts.keys())

In [15]:
def extract_text(data, text_column):
    '''
    Purpose: Helper function for bag_of_words
    Input: Dataset
    Output: array of email sets of words (sets don't allow duplicates)
    '''
    
    return(data.apply(lambda x:set(x[text_column].split(' ')), axis = 1))

In [13]:
def bag_of_words(data, **kwargs):
    '''
    Purpose: Converts a dataset to bag of words format.
    Input: Dataset
    Output: Bag of words version of the data
    '''
    
    emails = extract_text(data, )
        
    if len(kwargs) == 0:
        bag = extract_words_from_emails(emails)
    else:
        bag = extract_words_from_emails(extract_emails(kwargs['word_data']))
    
    word_occurence = words_in_texts(bag, emails)
    
    data = data.reset_index(drop = True)
    
    new_data = pd.DataFrame(data = word_occurence, columns = bag)
    new_data.insert(0, 'email_message', data['email'])
    new_data['spam_label'] = data['label']
        
    return(new_data)

In [14]:
bag_data = bag_of_words(data)

KeyError: 'email'

# Save dataset

In [29]:
try:
    data.to_csv(processed_root("01-poetryfoundation/poetry_foundation.csv"), index = False)
except FileNotFoundError:
    os.mkdir(processed_root("01-poetryfoundation"))
    data.to_csv(processed_root("01-poetryfoundation/poetry_foundation.csv"), index = False)